# Sugartrail 

In [1]:
from sugartrail import mapview, api, base
import ipywidgets as widgets
from IPython.display import display
import requests
import pandas as pd

In [ ]:
%%capture
network = base.Network()

1. Insert your [Companies House API](https://developer.company-information.service.gov.uk/how-to-create-an-application) key:

In [ ]:
API_input = widgets.Text(
    value='',
    placeholder='Insert API Key',
    disabled=False
)

company_text = widgets.Text(
    value='',
    placeholder='Insert Company ID',
    disabled=True
)

auth_status = widgets.HTML(
    value="",
)

auth_button = widgets.Button(description='Authenticate',button_style='success')
auth_button.on_click(lambda bt: auth())

init_button = widgets.Button(description='Initialise',button_style='success', disabled=True)
init_button.on_click(lambda bt: init_network()) 

def auth():
    auth_button.disabled=True
    API_input.disabled=True
    api.basic_auth.username = API_input.value
    if api.test():
        auth_status.value = u'\u2705: Login successful'
        company_text.disabled = False
        init_button.disabled = False
    else:
        auth_button.disabled=False
        API_input.disabled=False
        auth_status.value = u'\u274c: Invalid API key'

display(API_input, auth_button, auth_status)

2. Insert the unique company registration number (CRN) for a company you would like to investigate:

<!-- Insert a valid [Companies House Public Data API key](https://developer.company-information.service.gov.uk/get-started/) as `username` string value below. If you don't want to use the API and would prefer loading a pre-built network, uncomment and run the cell below and then run the final cell to build and load the map.  -->

In [ ]:
init_status = widgets.HTML(
    value="",
) 

depth_selector = widgets.BoundedIntText(
    value=1,
    min=1,
    max=5,
    step=1,
    disabled=True
)

generate_network_button = widgets.Button(description='Build Network',button_style='success', disabled=True)
generate_network_button.on_click(lambda bt: generate_network()) 

def init_network():
    init_button.disabled=True
    company_text.disabled=True
    api.basic_auth.username = API_input.value
    response = api.get_company(str(company_text.value))
    if response:
        network.company_id = str(company_text.value)
        init_status.value = u'\u2705: Initialisation successful for ' + str(response['company_name']) 
        depth_selector.disabled = False
        generate_network_button.disabled = False
    else:
        init_button.disabled=False
        company_text.disabled=False
        auth_button.disabled=False
        API_input.disabled=False
        init_status.value = u'\u274c: Initialisation Failed. No records for company: ' + str(company_text.value) + ' found.'

display(company_text, init_button, init_status)

3. Select the depth of the network you would like to build:

In [ ]:
build_status = widgets.HTML(
    value="",
)

build_map_button=widgets.Button(description='Build Map',button_style='success', disabled=True)
build_map_button.on_click(lambda bt: generate_map()) 

def generate_network():
    with output_box:
        depth_selector.disabled = True
        generate_network_button.disabled = True
        network.perform_hop(depth_selector.value + 1)
        network.run_map_preprocessing()
        build_map_button.disabled = False
        
    
output_box = widgets.Output()
display(depth_selector, generate_network_button, build_status, output_box)

4. Visualise network on a map:

In [ ]:
map_container = widgets.HTML(
    value="",
)

map_data,path_table = mapview.build_map(network, clear_widget=False) 
hbox_map = widgets.HBox([path_table])
vbox_map = widgets.VBox([map_data, hbox_map])

accordion_map = widgets.Accordion(children=[vbox_map])
accordion_map.set_title(0, 'Map')

tabs = ['Companies', 'Addresses', 'Officers', 'Company Details']
children = [widgets.Output() for tab in tabs]
tab = widgets.Tab()
tab.children = children
for i,title in enumerate(tabs):
    tab.set_title(i, title)

accordion_data = widgets.Accordion(children=[tab])
accordion_data.set_title(0, 'Data')

def generate_map():
    map_data,path_table = mapview.build_map(network, clear_widget=False) 
    hbox_map = widgets.HBox([path_table])
    vbox_map.children = [map_data, hbox_map]
    accordion_map.selected_index=0
    accordion_data.selected_index=0
    build_map_button.disabled = True
    with tab.children[0]:
        display(pd.DataFrame(network.company_ids))
    with tab.children[1]:
        display(pd.DataFrame(network.addresses))
    with tab.children[2]:
        display(pd.DataFrame(network.officer_ids))
    with tab.children[3]:
        display(pd.DataFrame(network.companies))   

display(build_map_button, map_container)

In [ ]:
accordion_map

In [ ]:
accordion_data